# Machine learning extractor -- Data Generation

idea:
learn a convolutional filter which extracts the features map on both target images and source inmages for the purpose that
1.  a small deviation from pose of target just bring samall change on distance of two feature map

2. target pose is the best optimized pose (golab minimum)

Dataset:
- target images from synthetic dataset with airplane pose (position, rotation)

- generate a batch of source images around each target image, with a slowly gradual increased deviation from target images

learning process:
- convolutional filters (one layer or several layers) will be appiled to a pair of images (a target image, and a source image near the target image) -> output two digits (or two vectors?)

- final output is distance between the two oupts,  

- ground truth can be acquired by calculating the discatance between
  souce image pose and target image pose ( how to design the distance of two pose? )

Potential issue:
- the alogrithm could be material-dependent. which mean it maybe doesn't work for real airplane image in real world which have different material, in spite of the same shape .



### 1. Data preparation
- read images and their poses from synthetic dataset as target images.

- for each targe image, generate a batch of source images around them
  
  optional idea:
  -  position of source images could be randon positions in the shpere whith target position as center and small radius, 
  - rotation of source images could be random rotation which are slightly deviated from target rotation ( better to use euler angle?)

- oragnize the dataset in the form of pairs 
  - each pair of images consist of one target image and a nearyby source image

In [1]:
  from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyrender
!pip install --upgrade redner-gpu
!pip install torchsummary

     |████████████████████████████████| 1.2 MB 7.4 MB/s 
     |████████████████████████████████| 2.2 MB 46.4 MB/s 
     |████████████████████████████████| 640 kB 62.7 MB/s 
     |████████████████████████████████| 890 kB 48.4 MB/s 
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745210 sha256=e4e5c3fc8e309b022ba98b305f1782791c19e651aaa2c005712bbad7f57c0b91
  Stored in directory: /root/.cache/pip/wheels/c6/83/cb/af51a0c06c33d08537b941bbfc87469e8a3c68d05f77a6a212
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.5
    Uninstalling PyOpenGL-3.1.5:
      Successfully uninstalled PyOpenGL-3.1.5
     |████████████████████████████████| 31.8 MB 1.2 MB/s 


In [3]:
import pyredner
import torch
import redner
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pyrender
import trimesh.transformations as transformations
import os
import h5py
import math
from matplotlib.pyplot import figure
from PIL import Image
from torchsummary import summary
import pandas as pd
import random
import cv2
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import time
# install BDH library:  !wget https://raw.githubusercontent.com/sfu-bigdata/range-driver/public/range_driver/dict_utils.py
from dict_utils import *
from utils_pose_transf import *
from utils_dataset_tools import *

ModuleNotFoundError: ignored

In [ ]:
#=================images generation
# load DC3 Model
def load_3D_model(mode_3D_path, env_path=None ):
    material_map, mesh_list, light_map = pyredner.load_obj(mode_3D_path)
    #airplane_objects = pyredner.load_obj(mode_3D_path, return_objects=True)

    # compute  mesh normal
    for _, mesh in mesh_list:
        mesh.normals = pyredner.compute_vertex_normal(mesh.vertices, mesh.indices)

    # # Setup camera for airplane (initialization)
    #cam = pyredner.automatic_camera_placement(objects, resolution=(512, 512))
    cam = pyredner.Camera(position = torch.tensor([-18.4245, 0, 5.68514]),   
                        look_at = torch.tensor([0.0, 0.0, 0.0]),
                        up = torch.tensor([0.0, 1.0, 0.0]),               
                        fov = torch.tensor([45.0]), # in degree
                        clip_near = 1e-2, 
                        resolution = (256, 256),
                        fisheye = False)      
    # Setup materials
    material_id_map = {}
    materials = []
    count = 0
    for key, value in material_map.items():
        #print(key)
        material_id_map[key] = count
        count += 1
        materials.append(value)

    # Setup geometries
    shapes = []
    for mtl_name, mesh in mesh_list:
        shapes.append(pyredner.Shape(\
            #vertices = mesh.vertices/100.0,    # keep unit consistent
            vertices = mesh.vertices,
            indices = mesh.indices,
            uvs = mesh.uvs ,
            normals = mesh.normals,
            material_id = material_id_map[mtl_name] if mtl_name is not None else 0
            #material_id = material_id_map[mtl_name]
            ))
            
    #-------------------------------------------------------------------------------
    #Adjust the pose of airplane
    #we can set tranalation and rotation for the airplane object here, 
    airplane_r = .0  #degree
    airplane_t = torch.tensor([.0, .0, .0])

    #rotate around x axis with some degrees
    radian_angle =  (airplane_r)/180*math.pi
    e_rotation = torch.tensor([radian_angle, 0.0, 0.0])
    rotation_matrix = pyredner.gen_rotate_matrix(e_rotation.cuda())

    t_vertices = torch.tensor([])
    for shape in shapes:
      v = shape.vertices
      t_vertices = torch.cat([t_vertices.cuda(), v])

    print('shape of t_vertices : ', t_vertices.shape)
    center = torch.mean(t_vertices, 0)

    for i in range(len(shapes)):
      shapes[i].vertices = (shapes[i].vertices -center)@ torch.t(rotation_matrix) + center + airplane_t.cuda()
    ##====================================================--------------------------

    ## set envmap (if you need to use)
    envmap = None
    if env_path: 
      envmap = pyredner.imread(env_path)
      if pyredner.get_use_gpu():
          envmap = envmap.cuda(device = pyredner.get_device())
          envmap = pyredner.EnvironmentMap(envmap)
    #--------------------------------------------------------------
   
    # We don't setup any light source here
    # put everything together  Construct the scene
    scene = pyredner.Scene(cam, shapes, materials, [],envmap = envmap)
    #scene = pyredner.Scene(cam, shapes, materials, area_lights = [], envmap = None)

    # Serialize the scene
    # Here we specify the output channels as "depth", "shading_normal"
    scene_args = pyredner.RenderFunction.serialize_scene(\
        scene = scene,
        num_samples = 16,
        max_bounces = 0,
        channels = [redner.channels.depth,            ## parameter "channel" specify which images you want to render
                    redner.channels.shading_normal])

    # Render the scene as our target image.
    render = pyredner.RenderFunction.apply
    # Render. The first argument is the seed for RNG in the renderer.
    img = render(0, *scene_args)
    test_normal = img[:, :, 1:4]
    plt.imshow(test_normal.cpu().detach().numpy())

    return [cam, shapes, materials, envmap]

In [ ]:
def render_sources(rendering_elements, data_s, dir_s, cam_resolution, fov, overwrite): 
    cam, shapes, materials, envmap = rendering_elements

    save_dir = os.path.join(dir_s, data_s[0])
    if os.path.exists(save_dir):
      if overwrite:
        print(f"Image overwrites: {save_dir}")
      else:
        print(f"Image already exists, skip generation:  {save_dir}")
    
    #convert translation and quaternion rotation to lootat matrix
    t_xyz = data_s[1:4]
    q_wxyz = data_s[4:]
    cam_matrix = qt_to_lookat(t_xyz, q_wxyz)

    cam.cam_to_world = torch.tensor(cam_matrix, dtype=torch.float32, device=pyredner.get_device())
    cam.resolution = cam_resolution 
    cam.fov = torch.tensor([fov])

    # We don't setup any light source here
    # put everything together  Construct the scene
    scene = pyredner.Scene(cam, shapes, materials, [],envmap = envmap)

    # scene_args
    scene_args = pyredner.RenderFunction.serialize_scene(scene = scene, num_samples = 16, \
            max_bounces = 0, channels = [redner.channels.depth, redner.channels.shading_normal])
    
    # Render the scene as our target image.
    render = pyredner.RenderFunction.apply
    # Render. The first argument is the seed for RNG in the renderer.
    img = render(0, *scene_args) 

    # Save the images.
    #print(img.size())
    depth = img[:, :, 0]
    normal = img[:, :, 1:4]

    
    pyredner.imwrite(normal.cpu(), save_dir)

In [ ]:
class Training_data_gene():
      def __init__(self, strategy_pose):
        self.strategy_pose = strategy_pose


      #prepare image data and annotation
      def gen_groups_sequence(self, rendering_elements, new_meta_fname, meta_dir, metadata_name, dir_s, cam_params, sample_p):
          # generate new metadata file: 
          # meta_dir: where original meta text stored
          # metadata_name: original meta data file name, original metadata file will be break down to train, val, test metadata file
          # "dir_s", directory where source images stored
          # sample_n: number of samples generated, sample_n[0]:number of samples with changed position, 
          # sample_n[1]: number of sample with chaged rotation, sample_n[2]: number of sample with chaged position and rotation, 
        
          cam_resolution, fov = cam_params
          # 0. read metadata from txt
          metadata_dir = os.path.join(meta_dir, metadata_name)
          new_meta_dir = os.path.join(meta_dir, new_meta_fname)

          if os.path.exists(new_meta_dir):
            os.remove(new_meta_dir)

          with open(os.path.join(metadata_dir),'r') as f, open(os.path.join(new_meta_dir),'a') as n_meta_f:
              #next(f)
              #next(f)
              #next(f)
              count = 0      # number of target images 
              for line in f:
                count += 1
                fname, p0, p1, p2, p3, p4, p5, p6 = line.split()[:8]
                #rename the file
                idx = fname.find('rgb_')
                if idx > 0:
                  t_fname = fname[idx:]
                else:
                  t_fname = fname
                print("------",t_fname)
                
                #generate pose around target:
                t_p = [float(p0), float(p1), float(p2), float(p3), float(p4), float(p5), float(p6)]
                
                #oganize a sequence of poses with increasing perturbation
                s_group = self.interplate_poses_sequence(t_p, t_fname, sample_p)

                #generate training metadata and render images 
                #training metadata consist of pairs of target image and source image
                lines = []
                for data in s_group:
                ## organize the sequence:
                ## each row in group is a quence of source images
                  line = ""
                  pose_t = p0+'|'+p1+'|'+p2+'|'+p3+'|'+p4+'|'+p5+ '|'+p6
                  line  +=  pose_t
                  pose_s = ""
                  name_s = ""
                  for source_img in data:
                      
                      pose_s += "," + str(source_img[1]) + '|'+str(source_img[2])+'|'+str(source_img[3])+'|'+str(source_img[4])+ \
                                '|'+str(source_img[5])+'|'+str(source_img[6])+ '|'+str(source_img[7])

                      name_s += "," + source_img[0]

                      # render source image
                      render_sources(rendering_elements, source_img, dir_s, cam_resolution, fov, True)

                  # format of each line: t_name s_name1 s_name2 ...  s_pose1 spose2 ... \n
                  line =  t_fname  + name_s + ' ' + pose_t  + pose_s + '\n'
                  lines.append(line)            
                
                # save to file
                # form in new metadata fiel:  target_image_name, source_image_name, target_pose, source_pose
                n_meta_f.writelines(lines)
                print('--- ',t_fname, "  processed")

          #calculate the size of dataset
          dst_size = count*len(s_group)
          print(f'target images size: {count}')
          print(f'dataset size: {dst_size} sequence')

          return dst_size, count


      # generate deviation sequence 
      # return sequence in posive direction and negative direction
      def gen_devi_seq(devi_from_sd, sample_p):
        #devi_from_sd: list of deviations from function "make_normal_bounded"
          def postive(x):
            return x > 0 
          
          def negative(x):
            return x < 0

          samples_postive = []
          while len(samples_postive) < sample_p:
              s = np.random.choice(devi_from_sd, size=sample_p)
              s = list(filter(postive, s))
              n_add = max(sample_p - len(samples_postive), 0)
              samples_postive.extend(s[:n_add])

          samples_negative = []
          while len(samples_negative) < sample_p:
              s = np.random.choice(devi_from_sd, size=sample_p)
              s = list(filter(negative, s))
              n_add = max(sample_p - len(samples_negative), 0)
              samples_negative.extend(s[:n_add])
          
          return [0] + sorted(samples_postive), [0] + sorted(samples_negative, reverse=True)
            


      # Generate sequences of poses arround target
      # strategy define how to generate those sequences
      def interplate_poses_sequence(self, t_p, t_fname, sample_p, dev_p=[4, 4, 4], dev_r = [0.3, 0.3, 0.3]):
          # t_p  pose
          # t_fname  target image name
          # sample_p:  number of sampel for each sweep
          # dev_p: sample camera position deviation p_x, p_y, p_z from dev_p.  i.e. sample x from [ -dev_p[0], dev_p[0] ], y from  [ -dev_p[1], dev_p[1] ] z from [ -dev_p[2], dev_p[2] ] 
          # dev_r: sample deviation of rotation euler angle r_x, r_y, r_z from dev_r. 
          # return: s_group   which store sequence of sourc image info:  name and poses




          print("Data interplote strategy:", self.strategy_pose)
          #----------------------------------------------

          t_position = t_p[:3]
          t_quaternion = t_p[3:]

          # convert quaternion to urler for easiler interpolation around target rotation
          # _, t_euler = euler_from_quaternion(t_quaternion)
          t_euler = transformations.euler_from_quaternion(t_quaternion)
          # print("---before devi")
          # print("t_quaternion", t_quaternion)
          # print("t_euler", t_euler)

          # deviation form x, y, z
          #devi_p = [0.5, 0.5, 0.5]
          #devi_r = [10, 10, 10]
          
          s_group = []
          # set target pos as the first element
      
          # choice_p = np.linspace(-1 * dev_p[0], dev_p[0], 51)
          # choice_r = np.linspace(-1 * dev_r[1], dev_r[2], 51)
          #generate samples using normal distribution
          dev_p_x = make_normal_bounded(-dev_p[0], +dev_p[0], sigma=dev_p[0]*.2, nsamples=200)
          dev_p_y = make_normal_bounded(-dev_p[1], +dev_p[1], sigma=dev_p[1]*.2, nsamples=200)
          dev_p_z = make_normal_bounded(-dev_p[2], +dev_p[2], sigma=dev_p[2]*.2, nsamples=200)

          dev_r_x = make_normal_bounded(-dev_r[0], +dev_r[0], sigma=dev_r[0]*.2, nsamples=200)
          dev_r_y = make_normal_bounded(-dev_r[1], +dev_r[1], sigma=dev_r[1]*.2, nsamples=200)
          dev_r_z = make_normal_bounded(-dev_r[2], +dev_r[2], sigma=dev_r[2]*.2, nsamples=200)

          # generate sorted deviation ( positive and negative  separately)
          offset_px_postive, offset_px_negative = gen_devi_seq(dev_p_x, sample_p-1)
          offset_py_postive, offset_py_negative = gen_devi_seq(dev_p_y, sample_p-1)
          offset_pz_postive, offset_pz_negative = gen_devi_seq(dev_p_z, sample_p-1)

          offset_rx_postive, offset_rx_negative = gen_devi_seq(dev_r_x, sample_p-1)
          offset_ry_postive, offset_ry_negative = gen_devi_seq(dev_r_y, sample_p-1)
          offset_rz_postive, offset_rz_negative = gen_devi_seq(dev_r_z, sample_p-1)


          s_all = np.array(t_p).reshape(7,1) * np.ones((1, sample_p))

          
          def add_group(s_a, s_fname):
              s_pose = pd.DataFrame(s_a.T)
              s_pose['fname'] = s_fname
              s_pose_r = s_pose[['fname'] + s_pose.columns[:-1].tolist()]
              s_group.append(s_pose_r.to_numpy())

          def quaternion_offset_xyz(tg_euler, offset_r_euler_xyz):
            offset_x = offset_r_euler_xyz[0]
            offset_y = offset_r_euler_xyz[1]
            offset_z = offset_r_euler_xyz[2]
            
            q_after_offset = []

            for i in range(len(offset_x)):
                s_q = transformations.quaternion_from_euler( \
                                  tg_euler[0] + offset_x[i] ,\
                                  tg_euler[1] + offset_y[i] , \
                                  tg_euler[2] + offset_z[i] )
                q_after_offset.append(s_q)
            
            #print("q_after_offset ---", q_after_offset)
            return q_after_offset    

          def gen_data_by_stradegy(strategy_pose):
            

            for stgy in strategy_pose:
              #sub_name to identify the type of pose deviation
              #e.g. [1,1,-1,0,0,0] -> "ppn000"  where p:positve n:negative 0:no deviation
              dev_type = "".join(list(map(lambda st: 'p' if st > 0 else ('0' if st==0 else 'n'),stgy)))
              
              p_x, p_y, p_z, r_x, r_y, r_z = stgy
              #-------------------
              if p_x >= 0: 
                offset_px = np.array(offset_px_postive) * p_x
              elif p_x <0:
                offset_px = np.array(offset_px_negative) * abs(p_x)         
              #
              if p_y >= 0: 
                offset_py = np.array(offset_py_postive) * p_y
              elif p_y <0:
                offset_py = np.array(offset_py_negative) * abs(p_y)   
              #
              if p_z >= 0: 
                offset_pz = np.array(offset_pz_postive) * p_z
              elif p_z <0:
                offset_pz = np.array(offset_pz_negative) * abs(p_z) 
              #-------------------
              if r_x >= 0: 
                offset_rx = np.array(offset_rx_postive) * r_x
              elif r_x <0:
                offset_rx = np.array(offset_rx_negative) * abs(r_x)        
                #
              if r_y >= 0: 
                offset_ry = np.array(offset_ry_postive) * r_y
              elif r_y <0:
                offset_ry = np.array(offset_ry_negative) * abs(r_y)       
              #
              if r_z >= 0: 
                offset_rz = np.array(offset_rz_postive) * r_z
              elif r_z <0:
                offset_rz = np.array(offset_rz_negative) * abs(r_z) 
              
              # print("---------------------")
              # print("offset_px ", offset_px)
              # print("offset_py ", offset_py)
              # print("offset_pz ", offset_pz)
              # print("offset_rx  ", offset_rx)
              # print("offset_ry  ", offset_ry)
              # print("offset_rz  ", offset_rz)
            
              s_orignal = s_all.copy()
              s_orignal[0,:] += offset_px
              s_orignal[1,:] += offset_py
              s_orignal[2,:] += offset_pz
              r_xyz_offset = [offset_rx, offset_ry, offset_rz]
              offset_q = quaternion_offset_xyz(t_euler,  r_xyz_offset)
              s_orignal[3:,:] = np.array(offset_q).T 
              s_fname = list(map(lambda st: f"{t_fname[:-4]}_{dev_type}_{st:03}.jpg", np.arange(len(offset_px_postive))))
              #print(s_fname)
              add_group(s_orignal, s_fname)

          # Generate metadata by strategy
          gen_data_by_stradegy(strategy_pose)

      #-----------------------------------------------------------------------------------------------------
  
          #print(s_group)
          return s_group


          # convert datset to h5 format
      def gen_dataseth5_seq(self, trainval_h5name, training_t_dir, training_s_dir , dir_h5, 
                                meta_name, resolution, sub_dir_img_t, sub_dir_seg_t, source_mode,
                                dst_size, seq_s_num=10, greyScale=False, applySeg=True ):
          # create h5 file
          c = 1 if greyScale else 3
          h = resolution[0]
          w = resolution[1]

          fileh5_p = os.path.join(dir_h5, trainval_h5name) 

          meta_p = os.path.join(training_t_dir,meta_name)
          with h5py.File(fileh5_p,'w') as f_dst_h5, open(meta_p,'r') as f_ann:
              print('records size ----', dst_size)
              #contruct h5 file
              
              dst_im_t = f_dst_h5.create_dataset('Train_im_t', (dst_size,h,w,c), 'f')
              dst_pose_t = f_dst_h5.create_dataset('Train_p_t', (dst_size,7), 'f')
              
              #create dataser format for source image according to the number of source images sequence
              for i in range(1, seq_s_num+1):
                f_dst_h5.create_dataset('Train_im_s'+str(i), (dst_size,h,w,c), 'f')
                f_dst_h5.create_dataset('Train_p_s'+str(i), (dst_size,7), 'f')
              
              for row_id, line in enumerate(f_ann):
                img_sequence = line.strip().split()
                #print(img_pair)
                # targe img 
                img_seq_names = img_sequence[0].split(",")
                img_seq_poses = img_sequence[1].split(",")

                for i in range(len(img_seq_names)):
                  if i == 0: # target img
                    t_img_name =  img_seq_names[i][4:-4].zfill(5)+'.jpg' if source_mode == 'from_boeing' else img_seq_names[i]
                    img_t_path =  os.path.join(training_t_dir, sub_dir_img_t, t_img_name)
                    #seg img
                    seg_name = img_seq_names[i][4:-4].zfill(5) + '.png' if source_mode == 'from_boeing' else img_seq_names[i]
                    img_seg_path =  os.path.join(training_t_dir, sub_dir_seg_t, seg_name) 

                    #targetimg pose
                    t_pose = img_seq_poses[i].strip().split("|")
                    t = [float(t_pose[i]), float(t_pose[1]),float(t_pose[2]),float(t_pose[3]),float(t_pose[4]), float(t_pose[5]), float(t_pose[6])]

                    #read target imgs
                    #print("target path:", img_t_path)
                    img_t = Image.open(img_t_path)
                    img_seg = cv2.imread(img_seg_path)

                    if source_mode == "from_blender":
                      img_seg = img_seg/255

                    # convert to greyscale ?
                    if greyScale:
                      img_t = np.asarray(img_t.convert("L"))
                      # apply segmentation on target image
                      if applySeg:
                        img_t = img_t * img_seg[0]
                    else:
                      img_t = np.asarray(img_t)
                      if applySeg:
                        img_t = img_t * img_seg
                    
                    #resize target img as the same size of renderred source imgs
                    img_t = cv2.resize(img_t, (w,h))

                    # save target img
                    f_dst_h5['Train_im_t'][row_id] = img_t
                    f_dst_h5['Train_p_t'][row_id] = t
                    
                  else:  #source imgs
                    img_s_path =  os.path.join(training_s_dir, img_seq_names[i])
                    img_s = Image.open(img_s_path)
                    if greyScale:
                      img_s = np.asarray(img_s.convert("L"))
                    else:
                      img_s = np.asarray(img_s)

                    s_pose = img_seq_poses[i].strip().split("|")
                    s = [float(s_pose[0]), float(s_pose[1]),float(s_pose[2]),float(s_pose[3]),float(s_pose[4]), float(s_pose[5]), float(s_pose[6])]
                    #save source imgs
                    f_dst_h5['Train_im_s'+str(i)][row_id] = img_s
                    f_dst_h5['Train_p_s'+str(i)][row_id] = s

                print(f'%d processed'% row_id )





## Generate data (Traing, Validation, Test)

In [ ]:
# 1. generate sequence of new poses
# 2. render new images using new poses
# 3. Generat Dataset h5 files 

# datageneration strategy
strategy_pose = \
[ [ 1, 0, 0, 0, 0, 0]]     
  [1, 0.2, 0.2, 0, 0, 0],    
  [-1, 0, 0, 0, 0, 0],       
  [-1, -0.2, -0.2, 0, 0, 0], 

  [ 0, 1, 0, 0, 0, 0],
  [ 0.2, 1, 0.2, 0, 0, 0],
  [ 0, -1, 0, 0, 0, 0],
  [ -0.2, -1, -0.2, 0, 0, 0],

  [ 0, 0, 1, 0, 0, 0],
  [ 0.2, 0.2, 1, 0, 0, 0],
  [ 0, 0,-1, 0, 0, 0],
  [ -0.2, -0.2,-1, 0, 0, 0],


  [ 0, 0, 0, 1, 0, 0],
  [ 0, 0, 0, 1, 0.2, 0.2],
  [ 0, 0, 0,-1, 0, 0],
  [ 0, 0, 0,-1, -0.2, -0.2],

  [ 0, 0, 0, 0, 1, 0],
  [ 0, 0, 0, 0.2, 1, 0.2],
  [ 0, 0, 0, 0,-1, 0],
  [ 0, 0, 0, -0.2,-1, -0.2],

  [ 0, 0, 0, 0, 0, 1],
  [ 0, 0, 0, 0.2, 0.2, 1],
  [ 0, 0, 0, 0, 0, -1],
  [ 0, 0, 0, -0.2, -0.2, -1],

  [1, 1, 1, 0, 0, 0] ]



original_img_size = (600, 800)
wh_ratio = 800/600

#set source image size, not too big for training
render_width = 400
rendered_size = (int(render_width/wh_ratio), render_width)   # keep the ratio consistent with original's
fov  = 45.0
cam_params = [rendered_size, fov]

meta_dir = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/dataset/'
train_meta_name = 'train_metadata.txt'
test_meta_name = 'test_metadata.txt'
val_meta_name = 'val_metadata.txt'

#the directon where renderred source images are stored
dir_s = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/dataset/train_seq/'

training_t_dir = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/dataset/'
training_s_dir = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/dataset/train_seq/'
meta_pair_name = 'metadata_pairs.txt'
dir_h5 = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/dataset/trainval_h5/'

# number of source image in each sequence
sample_p = 10
#s_size = sum(sample_n)

train_seq_fname = 'metadata_train_sequence.txt'      # dataset training sequence
test_seq_fname = 'metadata_test_sequence.txt'  # dataset testing sequence
val_seq_fname = 'metadata_val_sequence.txt'

#----3D model
model_3D_base_dir = '/content/drive/MyDrive/BigDataHub/Differentiable_Render/DC3_model'
mode_3D_name = 'dc3.obj'
env_file_name = 'sunsky.exr'

#-----store in h5 file--------
# training data
sub_dir_img_t = 'Blender_images/render_imgs'
sub_dir_seg_t = 'Blender_images/seg_imgs'
source_mode = 'from_blender'

h5_name = 'train_learn_seq.h5'
val_h5_name = 'val_learn_seq.h5'
h5_name = 'test_learn_data.h5'


### load 3D model

In [ ]:
#load 3D model 
mode_3D_path = os.path.join(model_3D_base_dir, mode_3D_name)
env_path = os.path.join(model_3D_base_dir, env_file_name)

rendering_elements = load_3D_model(mode_3D_path)

### intilize Data generate engineer

In [ ]:
# intilize class
data_generation = Training_data_gene(strategy_pose)

### Generate training data

In [ ]:
#generate metadata and images
dst_size, count_t = data_generation.gen_groups_sequence(rendering_elements, train_seq_fname, meta_dir, train_meta_name, dir_s, cam_params, sample_p)

In [ ]:
#convert training data to memory mapping h5 file to accelarate the training speed
data_generation.gen_dataseth5_seq(h5_name, training_t_dir, training_s_dir, dir_h5, train_seq_fname, rendered_size, 
                  sub_dir_img_t, sub_dir_seg_t, source_mode, dst_size, sample_p)

## Generate validate data

In [ ]:
#generate training img
#dst_n = gen_groups(meta_dir, meta_name, dir_s)
val_dst_size, val_count_t = data_generation.gen_groups_sequence(rendering_elements, val_seq_fname, meta_dir, val_meta_name, dir_s, rendered_size, sample_p)

In [ ]:
# generate h5
data_generation.gen_dataseth5_seq(rendering_elements, val_h5_name, training_t_dir, training_s_dir, dir_h5, val_seq_fname, rendered_size, 
                  sub_dir_img_t, sub_dir_seg_t, source_mode, val_dst_size, sample_p)


### Generate Test Data

In [ ]:
#generate training img
#dst_n = gen_groups(meta_dir, meta_name, dir_s)
dst_tet_size, count_t = data_generation.gen_groups_sequence(rendering_elements, test_seq_fname, meta_dir, test_meta_name, dir_s, rendered_size, sample_p)

In [ ]:
# generate h5
data_generation.gen_dataseth5_seq(h5_name, training_t_dir, training_s_dir, dir_h5, test_seq_fname, rendered_size, 
                  sub_dir_img_t, sub_dir_seg_t, source_mode, dst_tet_size, sample_p)